# Importing libraries

In [1]:
!ls

100GO_bonds_coverage.csv
aladdin_index_data_sample_cols-copy1.ipynb
Blackrock_Project_Code (1).ipynb
Carbon_Data_Analysis_MUB_Bmk.ipynb
Carbon_Data_by_GDP.csv
Carbon_Data_Carbon_Monitor_BEA_Analysis_MUB_Bmk_Working.ipynb
Carbon_Data_EIA_MUB_Bmk_Per_Capita_Working.ipynb
Carbon_Data_EIA_MUB_Bmk_Working.ipynb
Carbon_Per_Capita.csv
charts&graphs.ipynb
Coding_stack.ipynb
Coding Stack_version2_with trend signal.ipynb
Coding Stack_version3_with zscored trend signal.ipynb
Consumption Energy Data.xlsx
Daily_carbonmonitor.csv
Datapoint_name_Revenue_bonds.csv
esg_data_prism_example.ipynb
ESG_Muni_MFRA.ipynb
ESG_Muni_Sample-Copy2.ipynb
ESG_Muni_Sample.ipynb
ESG_Muni_Sample-Orig.ipynb
ESG_Signal_Framework_Pankaj.ipynb
Green_Consumption_percent_signal.ipynb
Green_Consumption_percent_signal_with_q5_minus_benchmark.ipynb
Green_percent.csv
Green_percent_debt.ipynb
Green_Percent_Production.csv
Green_Policy_Rank_Dataset.csv
Green_policy_signal.ipynb
Green_policy_signal_with_benchmark_minus_q1-Copy1.ipynb

In [2]:
import pandas as pd 
import numpy as np 
import os
from google.colab import drive 
# drive.mount('/content/drive') ## use this when you need to mount the google drive
#Importing data for ICE index 
import pickle 
# Importing data for future manipulations 
from google.colab import files 
# uploaded = files.upload() ## use this when you need to upload the files in the workspace

# importing matlplotlib for plotting purposes
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder 
from collections import Counter
from imblearn.over_sampling import SMOTE 
from imblearn.under_sampling import RandomUnderSampler 
from imblearn.pipeline import Pipeline 
from sklearn.linear_model import LinearRegression 
from matplotlib import pyplot 

# After Rescaling Most important Features
from sklearn.preprocessing import MinMaxScaler 

import statsmodels.api as sm 
from collections import defaultdict 

# Downloading Data below and saving for later use

In [ ]:
# There are some changes to be done in the code to use this
#Importing data for ICE index 

import pickle 

if 'ICE_dataset.pkl' in listfiles:
  pkl_file = open('ICE_dataset.pkl', 'rb')
  ICE_dataset = pickle.load(pkl_file)
  pkl_file.close()

else:
  ICE_dataset = {} 
  for i in listfiles: 
    ICE_dataset[i[5:13]] = pd.read_excel(root_drive+'/ICE/'+i,skiprows=[0]) 
  # write python dict to a file
  output = open('ICE_dataset.pkl', 'wb') 
  pickle.dump(ICE_dataset, output) 
  output.close() 

# Main Functions

In [ ]:
# construct Green Energy signal 
def construct_green_energy_signal(data, columns = ['date','state_ticker','Green_percent'],new_column): 
    Pivot_scores_df = data[columns].copy() 
    Green_energy_scores = get_xs_scores(Pivot_scores_df.pivot_table(index = 'date', columns = 'state_ticker', values = columns[2])) 
    data.set_index(list(columns[:2]), inplace = True) 
    data[new_column] = Green_energy_scores.stack() 
    return data 

def get_xs_scores(in_data, wins_upp = 3, wins_low = -3, percent = False): 
    # winsorize 
    in_data = in_data.clip(in_data.mean(1) + wins_low * in_data.std(1), in_data.mean(1) + wins_upp * in_data.std(1), axis = 0) 
    # XS z-score 
    if percent: 
        return in_data.rank(1, pct=True).subtract(in_data.rank(1, pct=True).mean(1), 0) 
    else: 
        return in_data.subtract(in_data.mean(1), 0).divide(in_data.std(1), 0) 
    
def create_quintile(data, signal): 
    data[signal] = data[signal].fillna(0.0)  
    data['quintile'] = data.groupby(level = 0, group_keys = False).apply(lambda x: pd.qcut(x[signal].rank(method='first'), 5, labels = range(1, 6))).astype(int)     
    return data

In [ ]:
def quintile_analysis_benchmark_remove_quintile(data, signal, bmk_ret, quintile): 
    data = create_quintile(data, signal) 
    
    # removing securities in the first quintile 
    data.drop(data[data['quintile'] == quintile].index, inplace = True) 
    data.reset_index(inplace = True)
    data = construct_green_energy_signal(data,signal) 
    data = create_quintile(data, signal) 
    tot_ret_q = data.reset_index(level=0).groupby(['date', 'quintile']).apply(lambda x: (x['bmk_wgt'] * x['next_ret']).sum() / x.loc[x['next_ret'].notnull(), 'bmk_wgt'].sum()).unstack() 
    tot_ret_q['bmk'] = bmk_ret 
    
    signal_stats = pd.DataFrame(index = ['Total Return', 'Total Vol', 'SR', 'Alpha', 'Beta', 'Beta Adjusted IR'], columns = [1, 2, 3, 4, 5,'bmk']) 
    signal_stats.loc['Total Return'] = tot_ret_q.mean() * 12 
    signal_stats.loc['Total Vol'] = tot_ret_q.std() * np.sqrt(12) 
    # assume risk free rate of 0... need to add rf rate 
    signal_stats.loc['SR'] = (tot_ret_q.mean() * 12) / (tot_ret_q.std() * np.sqrt(12)) 
    resid_ret_q = pd.DataFrame(index = tot_ret_q.index, columns = [1, 2, 3, 4, 5,'bmk']) 
    for i in range(1, 6): 
        tmp_reg = sm.OLS(tot_ret_q[i], sm.add_constant(tot_ret_q['bmk']), missing = 'drop').fit() 
        resid_ret_q[i] = tmp_reg.resid + tmp_reg.params['const'] 
        signal_stats.loc['Alpha', i] = resid_ret_q[i].mean() * 12 
        signal_stats.loc['Beta', i] = tmp_reg.params['bmk'] 
        signal_stats.loc['Beta Adjusted IR', i] = (resid_ret_q[i].mean() * 12) / (resid_ret_q[i].std() * np.sqrt(12))         
    return tot_ret_q, resid_ret_q, signal_stats 

In [ ]:
def stacked_plots(data,label = 'Index',ind_lvl = 'Industry Lvl 3 Desc',val = 'CA'):
  
  data_ind_l3_ret = data[['As of Date',ind_lvl,'% Mkt Value']].copy() 
  data_ind_l3_ret_grp = data_ind_l3_ret.groupby(['As of Date',ind_lvl]).sum() 
  data_ind_l3_ret_grp = data_ind_l3_ret_grp.reset_index( level = [0,1] ) 
  data_ind_l3_ret_grp_stacked = data_ind_l3_ret_grp.set_index(['As of Date', ind_lvl]).unstack([ind_lvl]) 
  data_ind_l3_ret_grp_stacked = data_ind_l3_ret_grp_stacked.reindex(columns = sorted(data_ind_l3_ret_grp_stacked.columns, key = lambda x: x[::-1])) 
  data_ind_l3_ret_grp_stacked.columns = data_ind_l3_ret_grp_stacked.columns.droplevel() 
  data_ind_l3_ret_grp_stacked.reset_index(inplace = True) 
  data_ind_l3_ret_grp_stacked.plot(x = 'As of Date', kind = 'bar', stacked = True, title = 'Stacked Bar '+ val +'_'+ label + ind_lvl)  
  ax1 = plt.axes()
  x_axis = ax1.axes.get_xaxis()
  x_axis.set_visible(False)
  plt.show()
  return


In [ ]:
# Add bar plots in the data 
def barplot():
    
    return

In [ ]:
def plot_feature_statistics(model,X,y):
    yhat = model.predict(X)
    SS_Residual = sum((y - yhat)**2)       
    SS_Total = sum((y-np.mean(y))**2)     
    r_squared = 1 - (float(SS_Residual))/SS_Total
    adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-X.shape[1]-1)
    print(r_squared, adjusted_r_squared) 
    return 

In [ ]:
def diff_org_years(x):
  if(0<=x<10):
    return '1-10'
  elif(10<=x<15):
    return '10-15'
  elif(15<=x<20):
    return '15-20'
  elif(20<=x<25):
    return '20-25'
  elif(25<=x<30):
    return '25-30'
  elif(30<=x<35):
    return '30-35'
  elif(35<=x<40):
    return '35-40'
  elif(40<=x<45):
    return '40-45'
  elif(45<=x<50):
    return '45-50'
  elif(50<=x<55):
    return '50-55'

def liquidity_analysis(data):
  data['Diff_years'] = (data['Maturity Date'] -  data['As of Date'])/365 
  data['Diff_years'] = data['Diff_years'].dt.days 
  data['Diff_years_interval'] = data['Diff_years'].apply(lambda x : diff_org_years(x)) 
  data['Maturty_time_weight'] = data['% Mkt Value']*data['Diff_years'] 
  data_grp_interval_sum = data[data['As of Date']==data['As of Date'][0]].groupby(['Diff_years_interval']).sum() 
  print(data_grp_interval_sum.loc[:,['% Mkt Value','Face Value_LOC','Maturty_time_weight']]) 
  return
